# 1. Import libraries and data

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
#Python shortcut for importing files, create string & assign to variable 
path = r'/Users/Chloe/Documents/2. CareerFoundry/2. Data Immersion pt 1/Achievement 4/07-2022 Instacart Basket Analysis'

In [3]:
path

'/Users/Chloe/Documents/2. CareerFoundry/2. Data Immersion pt 1/Achievement 4/07-2022 Instacart Basket Analysis'

In [4]:
#import the latest file using the os.path.join() function
df = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'final_instacart_data.pkl'))

# 2. Check dataframe

In [5]:
df.shape

(32404859, 34)

In [6]:
df.head()

,order_id,user_id,order_number,orders_day_of_week,hour_of_day_ordered,days_since_prior_order,new_customer,product_id,add_to_cart_order,reordered,...,frequency_flag,first_name,surname,gender,state,age,date_joined,#_of_dependants,marital_status,income
0,2539329,1,1,2,8,NaN,True,196,1,0,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,2,3,7,15.0,False,196,1,1,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,3,3,12,21.0,False,196,1,1,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,4,4,7,29.0,False,196,1,1,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,5,4,15,28.0,False,196,1,1,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 34 columns):
 #   Column                         Dtype   
---  ------                         -----   
 0   order_id                       int64   
 1   user_id                        object  
 2   order_number                   int64   
 3   orders_day_of_week             int64   
 4   hour_of_day_ordered            int64   
 5   days_since_prior_order         float64 
 6   new_customer                   bool    
 7   product_id                     int64   
 8   add_to_cart_order              int64   
 9   reordered                      int64   
 10  product_name                   object  
 11  aisle_id                       int64   
 12  department_id                  int64   
 13  prices                         float64 
 14  _merge                         category
 15  price_range_loc                object  
 16  busiest_day                    object  
 17  busiest_days             

# 3. Security checks

The only PII in this data set would be customer names. Because we have customer ID's we can remove the names in order to protect their privacy.

In [8]:
#drop first and last name columns. _merge can also be dropped

df2 = df.drop(columns=['first_name', 'surname', '_merge'])

In [9]:
#confirm that the columns dropped successfully
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 31 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   order_id                       int64  
 1   user_id                        object 
 2   order_number                   int64  
 3   orders_day_of_week             int64  
 4   hour_of_day_ordered            int64  
 5   days_since_prior_order         float64
 6   new_customer                   bool   
 7   product_id                     int64  
 8   add_to_cart_order              int64  
 9   reordered                      int64  
 10  product_name                   object 
 11  aisle_id                       int64  
 12  department_id                  int64  
 13  prices                         float64
 14  price_range_loc                object 
 15  busiest_day                    object 
 16  busiest_days                   object 
 17  busiest_period_of_day          object 
 18  

# Create region flags

In [11]:
#using Wikipedia source provided, establish regions based on states to start creating new region column

Northeast = ['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 
             'Connecticut', 'New York', 'Pennsylvania', 'New Jersey']

Midwest = ['Wisconsin', 'Michigan', 'Illinois', 'Indiana', 'Ohio',
          'North Dakota', 'South Dakota', 'Nebraska', 'Kansas',
          'Minnesota', 'Iowa', 'Missouri']

South = ['Delaware', 'Maryland', 'District of Columbia', 'Virginia', 'West Virginia',
         'North Carolina', 'South Carolina', 'Georgia', 'Florida', 'Kentucky', 'Tennessee',
         'Mississippi', 'Alabama', 'Oklahoma', 'Texas', 'Arkansas', 'Louisiana']

West = ['Idaho', 'Montana', 'Wyoming', 'Nevada', 'Utah', 'Colorado', 'Arizona',
        'New Mexico', 'Alaska', 'Washington', 'Oregon', 'California', 'Hawaii']

In [13]:
#use the .loc function to create new region column
df2.loc[df2['state'].isin(Northeast), 'region'] = 'Northeast'

In [14]:
df2.loc[df2['state'].isin(Midwest), 'region'] = 'Midwest'

In [15]:
df2.loc[df2['state'].isin(South), 'region'] = 'South'

In [16]:
df2.loc[df2['state'].isin(West), 'region'] = 'West'

In [17]:
#check value counts for each region

df2['region'].value_counts(dropna=False)

South        10791885
West          8292913
Midwest       7597325
Northeast     5722736
Name: region, dtype: int64

In [18]:
10791885+8292913+7597325+5722736

32404859

Confirmed that every record was assigned a region

# Compare regional spending differences

In [19]:
# Use crosstab to see the frequency of low and high spending customers in each region

cross = pd.crosstab(df2['region'], df2['spending_flag'], dropna=False).sort_index()

In [20]:
cross

spending_flag,High spender,Low spender
region,,
Midwest,155975,7441350
Northeast,108225,5614511
South,209691,10582194
West,160354,8132559


The South has the highest number of "high spenders" while the Northeast has the least amount

In [21]:
# Use crosstab to see the customer frequency per region

cross2 = pd.crosstab(df2['region'], df2['frequency_flag'], dropna=False).sort_index()

In [22]:
cross2

frequency_flag,Frequent customer,Non-frequent customer,Regular customer
region,,,
Midwest,5017709,863419,1716197
Northeast,3804566,630182,1287988
South,7192372,1215088,2384420
West,5545206,927748,1819959


The South has the most frequent customers, and the Northeast has the lowest amount of frequent customers

In [23]:
#Look at which region has the most states allotted

df2[['region', 'state']].groupby('region').agg({'state': pd.Series.nunique,}).sort_values(by='state', ascending=False)

,state
region,
South,17
West,13
Midwest,12
Northeast,9


Important to note that the South has high frequency and high spending probably because it has the most states allotted to its region (almost double as many as Northeast)


In [25]:
# Consider region and user_id to see how many customers are in each region

df2[['region', 'user_id']].groupby('region').agg({'user_id': pd.Series.nunique,}).sort_values(by='user_id',ascending=False)

,user_id
region,
South,68737
West,52565
Midwest,48519
Northeast,36388


Important to note that the South has the most customers and Northeast has the least

# Create exclusion flag for low-activity customers

Instacart CFO isn’t interested in customers who don’t generate much revenue for the app. Create an exclusion flag for low-activity customers (customers with less than 5 orders) and exclude them from the data. Make sure you export this sample.

In [27]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 32 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   order_id                       int64  
 1   user_id                        object 
 2   order_number                   int64  
 3   orders_day_of_week             int64  
 4   hour_of_day_ordered            int64  
 5   days_since_prior_order         float64
 6   new_customer                   bool   
 7   product_id                     int64  
 8   add_to_cart_order              int64  
 9   reordered                      int64  
 10  product_name                   object 
 11  aisle_id                       int64  
 12  department_id                  int64  
 13  prices                         float64
 14  price_range_loc                object 
 15  busiest_day                    object 
 16  busiest_days                   object 
 17  busiest_period_of_day          object 
 18  

In [28]:
# Create variable for # of orders made by each user.
# The count of user_id would be the number of orders that a user has made 

df2['total_orders']=df2.groupby(['user_id']).user_id.transform('count')

In [30]:
# Create exclusion flag by counting the amount of times user_id shows up in dataframe

results =[]
for total in df2['total_orders']:
    if total < 5:
        results.append('Low Activity')
    else:
        results.append('Regular Activity')

In [31]:
#add column to df2

df2['activity_flag']=results

In [33]:
#check counts

df2['activity_flag'].value_counts(dropna=False)

Regular Activity    32401975
Low Activity            2884
Name: activity_flag, dtype: int64

In [34]:
# Create subset of low activity customers

low_activity_customers= df2[df2['activity_flag']=='Low Activity']

In [35]:
low_activity_customers.shape

(2884, 34)

In [36]:
# Export low activity subset

low_activity_customers.to_csv(os.path.join(path, '02 Data','Prepared Data', 'low_activity_customers.csv'))

In [37]:
# Drop low activity customer records from df2
df2.drop(df2[df2['activity_flag']=='Low Activity'].index, inplace=True)

In [38]:
# Double check counts 
df2['activity_flag'].value_counts()

Regular Activity    32401975
Name: activity_flag, dtype: int64

In [39]:
df2.head(10)

,order_id,user_id,order_number,orders_day_of_week,hour_of_day_ordered,days_since_prior_order,new_customer,product_id,add_to_cart_order,reordered,...,gender,state,age,date_joined,#_of_dependants,marital_status,income,region,total_orders,activity_flag
0,2539329,1,1,2,8,NaN,True,196,1,0,...,Female,Alabama,31,2/17/2019,3,married,40423,South,59,Regular Activity
1,2398795,1,2,3,7,15.0,False,196,1,1,...,Female,Alabama,31,2/17/2019,3,married,40423,South,59,Regular Activity
2,473747,1,3,3,12,21.0,False,196,1,1,...,Female,Alabama,31,2/17/2019,3,married,40423,South,59,Regular Activity
3,2254736,1,4,4,7,29.0,False,196,1,1,...,Female,Alabama,31,2/17/2019,3,married,40423,South,59,Regular Activity
4,431534,1,5,4,15,28.0,False,196,1,1,...,Female,Alabama,31,2/17/2019,3,married,40423,South,59,Regular Activity
5,3367565,1,6,2,7,19.0,False,196,1,1,...,Female,Alabama,31,2/17/2019,3,married,40423,South,59,Regular Activity
6,550135,1,7,1,9,20.0,False,196,1,1,...,Female,Alabama,31,2/17/2019,3,married,40423,South,59,Regular Activity
7,3108588,1,8,1,14,14.0,False,196,2,1,...,Female,Alabama,31,2/17/2019,3,married,40423,South,59,Regular Activity
8,2295261,1,9,1,16,0.0,False,196,4,1,...,Female,Alabama,31,2/17/2019,3,married,40423,South,59,Regular Activity
9,2550362,1,10,4,8,30.0,False,196,1,1,...,Female,Alabama,31,2/17/2019,3,married,40423,South,59,Regular Activity


In [40]:
df2.shape

(32401975, 34)

In [43]:
# Export new dataframe that only includes regular activity customers for the Instacart CFO

df2.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'regular_activity_customers.pkl'))